In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
import stumpy
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import numpy.testing as npt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os
from vnstock import * #import all functions, including functions that provide OHLC data for charting
# from vnstock.chart import * # import chart functions

In [3]:
# df = stock_historical_data("HAH", "2020-01-01", "2023-10-10", "1D", "stock")
# fig = candlestick_chart(df, ma_periods=[50,200], show_volume=False, reference_period=300, figure_size=(15, 8), 
#                         title='VIC - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
#                         colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
# fig.show()

Sử dụng các tiêu chí lọc như sau để thiết lập tham số params.

- CANSLIM: epsGrowth1Year, lastQuarterProfitGrowth, roe, avgTradingValue20Day, relativeStrength1Month
- Giá trị: roe, pe, avgTradingValue20Day
- Cổ tức cao: dividendYield, avgTradingValue20Day
- Phá nền mua: avgTradingValue20Day, forecastVolumeRatio, breakout: 'BULLISH'
- Giá tăng + Đột biến khối lượng: avgTradingValue20Day, forecastVolumeRatio
- Vượt đỉnh 52 tuần: avgTradingValue20Day, priceBreakOut52Week: 'BREAK_OUT'
- Phá đáy 52 tuần: avgTradingValue20Day, priceWashOut52Week: 'WASH_OUT'
- Uptrend ngắn hạn: avgTradingValue20Day, uptrend: 'buy-signal'
- Vượt trội ngắn hạn: relativeStrength3Day,
- Tăng trưởng: epsGrowth1Year, roe, avgTradingValue20Day

In [4]:
def filter_stocks(param):
    df = stock_screening_insights (param, size=1700, drop_lang='vi')
    if len(df)!= 0:
        print(df.ticker.unique())
    return df

## canslim

In [5]:
canslim_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    "epsGrowth1Year": (25, 100),  # Minimum 1-year EPS growth
    "lastQuarterProfitGrowth": (25, 100),  # Minimum last quarter profit growth
    "roe": (20, 100),  # Minimum Return on Equity (ROE)
    # "avgTradingValue20Day": (50, 1000),  # Minimum 20-day average trading value
}
df = filter_stocks(canslim_params)
df.head()

""


In [6]:
params = {
            "exchangeName": "HOSE",
            "marketCap": (1_000, 200_000),
            'uptrend': 'buy-signal',
            'tcbsBuySellSignal':'BUY',
            # "avgTradingValue20Day": (10, 100_000),
            'priceNearRealtime': (15,80),
        }
df = stock_screening_insights (params, size=1700, drop_lang='vi')
df.head()

,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,uptrend.en,priceVsSMA5.en,priceVsSMA20.en,foreignTransaction.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma10.en,priceVsSma50.en,bollingBandSignal.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,tcbsRecommend.en
0,HPG,,156418,1.9,2.7,2.8,4.4,2.8,0.1,1.4,58.8,12.3,5.7,0.7,83.6,1.6,19.0,0.0,-5.53,25.09,-75.39,-4.01,12.62,7.04,0.6,505.66,545.58,655.72,55,77,60,74,412.75,26.9,55.8,25.19,66,5136,1,322,1.0,0.8,0.6,0.7,1.89,7.17,1.32,7.17,38.30,120.77,55,-30.9,-27.8,1870.7,-16.5,-21.2,NaN,-63.8,-8.5,-38.7,57.3,3474.8,HOSE,Basic Resources,Buy Signal,Price cross above SMA(5),Price cross above SMA(20),Buy < Sell,Buy,MACD Histogram < 0 and increase,Price cross above SMA(10),Price above SMA(50),Buy signal,Neutral,Yes,Price above SMA(100),NaN
1,NLG,,14314,7.1,2.9,2.4,4.2,3.8,0.2,1.7,44.4,16.5,10.0,0.6,22.7,1.6,29.4,1.6,-16.65,6.54,-48.05,-1.38,41.95,18.58,0.4,69.34,107.54,112.25,42,88,55,65,48.29,37.2,56.3,43.18,62,4745,1,1640,0.8,0.5,0.4,0.6,0.81,16.25,0.54,16.25,28.89,85.98,65,-20.9,-10.8,631.0,-59.5,-23.2,738.7,9.2,-8.7,-41.3,64.2,394.1,HOSE,Real Estate,Buy Signal,Price cross above SMA(5),Price above SMA(20),Buy < Sell,Buy,MACD Histogram < 0 and decrease,Price cross above SMA(10),Price above SMA(50),NaN,Neutral,Yes,Price above SMA(100),Wait to buy
2,TBC,,2445,15.8,3.3,4.2,3.6,4.8,0.1,0.1,NaN,74.2,13.6,0.1,14.0,2.2,8.1,10.6,46.61,16.19,92.11,14.24,45.70,12.78,0.1,0.07,0.11,0.08,17,45,65,70,0.00,38.5,51.9,1.56,49,0,1,2757,0.1,0.0,0.0,0.0,2.94,1.58,1.32,1.58,38.45,106.26,10,-2.0,-3.2,175.1,-56.8,-54.3,-88.9,-79.5,-3.8,-3.8,43.1,927.1,HOSE,Utilities,Buy Signal,Price cross above SMA(5),Price cross above SMA(20),NaN,Buy,MACD Histogram < 0 and increase,Price cross above SMA(10),Price cross above SMA(50),Buy signal,Neutral,Yes,Price above SMA(100),NaN


## Payback time

In [7]:
# https://www.youtube.com/watch?v=4YPyxfXML0A&t=1825s
paybacktime_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    'revenueGrowth5Year': (15,100),
    'pe':(10,20),
    "epsGrowth5Year": (10, 100),  # Minimum 1-year EPS growth
    "roe": (15, 100),  # Minimum Return on Equity (ROE)
    
    'priceNearRealtime': (15,80),
#     "lastQuarterProfitGrowth": (2, 100),  # Minimum last quarter profit growth
    # "avgTradingValue20Day": (10, 2000),  # Minimum 20-day average trading value
#     "breakout": 'BULLISH',  # Only buy stocks when the market is in an uptrend
}
df = filter_stocks(paybacktime_params)
df.head()

['DCM' 'TBC' 'TDM']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,foreignTransaction.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,uptrend.en,priceVsSMA5.en,priceVsSma10.en,bollingBandSignal.en,tcbsRecommend.en
0,DCM,,16941,16.8,3.3,4.3,4.0,5.0,0.1,1.3,59.6,54.0,17.9,1.1,10.5,1.8,5.7,12.4,61.35,22.61,136.69,46.58,5.89,2.45,0.0,72.42,98.23,109.07,84,80,76,51,88.82,32.0,53.7,11.07,73,378,0,0.0,3050,1.6,1.0,0.9,0.6,0.31,9.22,0.00,9.22,20.99,292.93,25,11.9,13.7,1614.7,-9.0,-19.4,-89.9,-69.4,-15.6,-23.3,55.6,603.6,HOSE,Chemicals,Price above SMA(20),Buy > Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN,NaN,NaN,NaN
1,TBC,,2445,15.8,3.3,4.2,3.6,4.8,0.1,0.1,NaN,74.2,13.6,0.1,14.0,2.2,8.1,10.6,46.61,16.19,92.11,14.24,45.70,12.78,0.1,0.07,0.11,0.08,17,45,65,70,0.00,38.5,51.9,1.56,49,0,1,NaN,2757,0.1,0.0,0.0,0.0,2.94,1.58,1.32,1.58,38.45,106.26,10,-2.0,-3.2,175.1,-56.8,-54.3,-88.9,-79.5,-3.8,-3.8,43.1,927.1,HOSE,Utilities,Price cross above SMA(20),NaN,Buy,MACD Histogram < 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),Buy Signal,Price cross above SMA(5),Price cross above SMA(10),Buy signal,NaN
2,TDM,,4100,15.3,2.9,3.8,3.4,4.2,0.1,0.1,57.1,85.6,2.4,0.2,13.1,1.9,11.6,0.0,14.84,19.44,-32.93,23.76,46.22,52.99,0.1,0.39,0.67,2.28,76,11,56,47,0.09,41.0,52.1,4.82,48,-117,1,NaN,3139,0.2,0.1,0.0,0.0,-0.73,-0.24,2.50,-0.24,21.35,147.94,70,-5.3,-8.6,313.9,-13.1,6.3,4.1,8.0,-4.7,-4.2,34.4,627.2,HOSE,Utilities,Price cross above SMA(20),NaN,Buy,MACD Histogram < 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,Price cross above SMA(5),Price cross above SMA(10),Buy signal,Wait to buy


## Tang Truong

In [8]:
tangtruong_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    "epsGrowth1Year": (10, 100),  # Minimum 1-year EPS growth
    "roe": (15, 100),  # Minimum Return on Equity (ROE)
    'priceNearRealtime': (15,80),
    # "avgTradingValue20Day": (10, 2000),  # Minimum 20-day average trading value
}
df = filter_stocks(tangtruong_params)
df.head()

['ACB' 'CAV' 'CTG' 'DBD' 'GAS' 'GMD' 'HAH' 'HDB' 'IMP' 'LIX' 'MBB' 'OPC'
 'PGD' 'PHR' 'REE' 'S4A' 'SAB' 'SCS' 'SHP' 'SMB' 'STB' 'TBC' 'TLG' 'TMP'
 'TPB' 'TRA' 'VFG' 'VGC' 'VIB' 'VPI']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,foreignTransaction.en,sarVsMacdHist.en,priceVsSMA5.en,tcbsRecommend.en,priceVsSma10.en,uptrend.en,bollingBandSignal.en,breakout.en
0,ACB,,85255,24.3,3.2,4.6,3.9,3.6,0.0,0.8,43.1,81.5,8.0,1.1,5.7,1.3,NaN,5.5,22.18,20.27,42.54,35.07,NaN,47.93,NaN,102.32,121.54,159.43,44,35,59,48,125.74,21.95,44.7,30.00,46,0,0.0,0.0,3839,1.6,1.2,0.8,1.0,-0.45,-0.45,0.00,-0.45,11.82,85.20,90.0,NaN,NaN,14909.0,18.8,15.9,12.6,-2.0,-10.0,-27.7,20.8,668.0,HOSE,Banks,Price below SMA(20),Sell,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAV,,3823,24.6,3.4,2.7,3.8,3.6,0.2,0.1,25.0,81.4,9.5,0.1,8.9,2.3,5.7,24.6,8.16,10.67,14.43,3.34,12.33,5.07,1.0,0.54,1.20,0.93,42,40,83,86,0.08,66.50,52.0,0.25,63,7,1.0,NaN,7500,0.2,0.1,0.1,0.1,-1.92,2.35,0.91,2.35,58.04,56.81,3.7,-40.4,-37.2,431.2,-9.0,-21.3,250.0,-11.0,-8.4,-5.2,68.8,440.0,HOSE,Industrial Goods & Services,Price above SMA(20),Neutral,MACD Histogram < 0 and decrease,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CTG,,141231,16.0,2.9,3.1,3.8,3.4,0.0,1.0,45.2,56.2,27.8,0.8,7.8,1.2,NaN,0.0,20.62,14.47,19.07,15.10,NaN,22.40,NaN,59.73,61.78,81.85,46,48,40,34,51.37,26.30,50.6,27.23,42,3832,0.0,0.0,3377,1.1,1.0,0.7,0.6,-0.71,4.96,0.00,4.96,5.15,46.06,15.0,NaN,NaN,18133.2,0.4,11.5,15.7,12.0,-12.7,-36.1,13.0,334.6,HOSE,Banks,Price above SMA(20),Sell,MACD Histogram < 0 and decrease,Price above SMA(50),Neutral,Yes,Price above SMA(100),Buy < Sell,Sell signal,NaN,NaN,NaN,NaN,NaN,NaN
3,DBD,,4378,21.2,2.8,3.0,3.3,4.8,0.2,0.1,61.6,64.4,51.6,0.4,15.1,3.1,13.9,0.0,-0.24,2.29,28.78,2.65,48.55,16.34,0.0,6.01,3.92,3.13,96,22,91,87,2.18,58.50,57.4,13.08,74,348,NaN,1.0,3866,0.4,0.6,0.7,0.4,3.17,-0.68,-0.68,-0.68,69.44,127.73,80.0,-0.5,-1.2,289.3,4.4,22.9,24.5,25.1,-2.5,-2.0,74.7,182.4,HOSE,Health Care,Price above SMA(20),Strong buy,MACD Histogram > 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Buy > Sell,Buy signal,NaN,NaN,NaN,NaN,NaN,NaN
4,GAS,,178457,20.5,2.8,3.0,4.5,5.0,0.0,0.7,59.9,6.3,0.2,0.9,14.7,2.9,10.4,4.4,27.51,9.32,70.63,4.95,16.64,10.74,0.1,38.35,40.08,40.87,59,29,51,13,36.70,77.50,43.4,2.80,38,-2045,1.0,NaN,5283,1.2,1.0,0.9,0.9,0.00,-2.02,1.31,-2.02,-13.34,-7.33,5.0,2.6,5.5,12133.1,-9.1,-13.1,-21.5,-38.0,-16.9,-36.3,5.2,269.2,HOSE,Utilities,Price below SMA(20),Sell,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),Buy > Sell,NaN,Price cross above SMA(5),NaN,NaN,NaN,NaN,NaN


## my params

In [9]:
my_params = {
    "exchangeName": "HOSE,HNX",
    "marketCap": (1_000, 200_000),
    "roe": (10, 100),  # Minimum Return on Equity (ROE)
    'pe': (10,20),
    'priceNearRealtime': (10,100),
    # "avgTradingValue20Day": (100, 1000),  # Minimum 20-day average trading value
#     'uptrend': 'buy-signal',
    'macdHistogram': 'macdHistLT0Increase',
    'strongBuyPercentage': (20,100),
    'relativeStrength3Day': (50,100),
}
df = filter_stocks(my_params)
df.head()

['CHP' 'CVT' 'IDC' 'LPB' 'TDM']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,foreignTransaction.en,tcbsRecommend.en,priceVsSMA5.en,priceVsSma10.en,bollingBandSignal.en
0,CHP,,3599,17.1,3.1,3.2,3.4,4.2,0.0,0.1,100.0,83.8,80.6,0.7,10.9,1.9,10.1,14.3,39.76,5.04,110.80,4.46,33.30,13.31,0.5,0.31,0.30,0.29,79,34,81,49,0.22,24.5,63.6,3.81,61,-11,3,NaN,2241,0.8,0.8,0.8,0.8,1.24,3.81,1.24,3.81,19.28,46.32,30.0,-16.7,-20.4,329.2,-42.7,-48.6,-81.3,-79.8,-1.8,0.8,20.4,886.9,HOSE,Utilities,Price above SMA(20),Buy,MACD Histogram < 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),NaN,NaN,NaN,NaN,NaN
1,CVT,,1174,11.1,2.0,2.3,3.0,2.4,-0.1,-0.1,90.0,24.1,170.1,3.1,12.0,1.3,8.6,0.0,40.08,11.35,0.79,-11.39,18.21,3.31,2.1,0.02,0.03,0.06,84,7,17,5,0.04,32.0,41.1,0.51,28,0,0,0.0,2659,3.1,1.9,0.7,0.9,-4.19,-3.03,0.00,-3.03,-30.21,78.92,48.9,-160.0,-56.2,97.6,-9.6,-3.3,-2.9,3.1,-30.2,-39.1,4.1,3492.2,HOSE,Construction & Materials,Price below SMA(20),Strong sell,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN,NaN,NaN,NaN
2,IDC,,16170,22.0,3.2,3.7,4.0,4.4,0.2,1.4,36.9,50.3,0.0,0.6,15.4,3.7,6.8,0.0,74.03,8.75,289.05,36.84,28.07,11.15,0.6,84.84,118.47,127.38,62,75,85,75,54.48,49.0,53.0,15.63,74,1374,0,0.0,3177,0.7,0.5,0.4,0.3,0.20,9.13,0.00,9.13,43.73,231.97,65.0,-12.8,-12.2,1048.5,-29.7,-27.2,-61.9,-62.6,-7.9,-40.7,78.0,285.9,HNX,Real Estate,Price above SMA(20),Buy,MACD Histogram < 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Buy < Sell,Wait to buy,NaN,NaN,NaN
3,LPB,,39132,14.5,2.7,3.7,3.1,1.7,0.2,1.1,28.9,37.7,10.8,0.4,10.8,1.5,NaN,0.0,40.99,22.66,56.97,9.18,NaN,31.88,NaN,43.80,37.57,39.10,78,41,54,88,18.51,15.3,52.9,3.16,65,285,0,0.0,1412,0.5,0.5,0.5,0.4,0.99,0.33,0.00,0.33,62.11,147.80,90.0,NaN,NaN,3612.3,-9.9,-23.5,0.6,-50.6,-7.5,-44.0,76.7,274.4,HOSE,Banks,Price below SMA(20),Buy,MACD Histogram < 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Buy < Sell,NaN,NaN,NaN,NaN
4,TDM,,4100,15.3,2.9,3.8,3.4,4.2,0.1,0.1,57.1,85.6,2.4,0.2,13.1,1.9,11.6,0.0,14.84,19.44,-32.93,23.76,46.22,52.99,0.1,0.39,0.67,2.28,76,11,56,47,0.09,41.0,52.1,4.82,48,-117,1,NaN,3139,0.2,0.1,0.0,0.0,-0.73,-0.24,2.50,-0.24,21.35,147.94,70.0,-5.3,-8.6,313.9,-13.1,6.3,4.1,8.0,-4.7,-4.2,34.4,627.2,HOSE,Utilities,Price cross above SMA(20),Buy,MACD Histogram < 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,Wait to buy,Price cross above SMA(5),Price cross above SMA(10),Buy signal


## Trend

In [10]:
trend_params = {
    "exchangeName": "HOSE,HNX",
    "marketCap": (1_000, 200_000),
    "epsGrowth5Year": (20, 100),  # Minimum 1-year EPS growth
#     "roe": (10, 100),  # Minimum Return on Equity (ROE)
#     'pe': (10,20),
#     'pb': (0,4),
    'priceNearRealtime': (10,100),
    'macdHistogram': 'macdHistLT0Increase',
    'strongBuyPercentage': (20,100),
    # "avgTradingValue20Day": (100, 100_000),  # Minimum 20-day average trading value
    # 'uptrend': 'buy-signal',
}
df = filter_stocks(trend_params)
df.head()

['ACB' 'BWE' 'DTD' 'EIB' 'HDB' 'IDC' 'MBB' 'MSB' 'SHB' 'STB' 'TCB' 'TDM'
 'TPB' 'VHC' 'VIB']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,priceVsSMA5.en,foreignTransaction.en,tcbsRecommend.en,priceVsSma10.en,bollingBandSignal.en
0,ACB,,85255,24.3,3.2,4.6,3.9,3.6,0.0,0.8,43.1,81.5,8.0,1.1,5.7,1.3,NaN,5.5,22.18,20.27,42.54,35.07,NaN,47.93,NaN,102.32,121.54,159.43,44,35,59,48,125.74,21.95,44.7,30.00,46,0,0,0.0,3839,1.6,1.2,0.8,1.0,-0.45,-0.45,0.00,-0.45,11.82,85.20,90,NaN,NaN,14909.0,18.8,15.9,12.6,-2.0,-10.0,-27.7,20.8,668.0,HOSE,Banks,Price below SMA(20),Sell,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN,NaN,NaN,NaN
1,BWE,,8228,15.5,2.7,3.9,3.8,3.2,0.0,0.1,41.3,51.4,0.3,0.1,11.9,1.8,9.2,0.0,11.70,14.17,-0.76,29.18,43.64,21.06,1.0,2.60,2.72,3.51,20,12,28,24,0.39,42.65,39.6,17.05,21,-58,1,NaN,3599,0.2,0.1,0.1,0.1,1.07,-3.07,1.07,-3.07,0.90,106.61,30,-56.8,-41.2,694.3,-2.3,3.5,-10.0,3.6,-14.0,-28.6,4.5,208.6,HOSE,Utilities,Price below SMA(20),Sell,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),Price cross above SMA(5),Buy > Sell,Wait to buy,NaN,NaN
2,DTD,,1135,25.0,3.6,4.3,3.5,5.0,0.4,2.0,30.6,39.4,0.0,0.6,5.8,1.2,2.5,0.0,-16.06,9.78,-11.00,27.19,30.11,19.38,0.0,12.95,17.66,19.01,75,91,33,93,9.38,23.00,51.0,0.24,73,-19,1,NaN,3993,0.8,0.6,0.5,0.5,-0.86,10.05,1.32,10.05,81.16,148.24,55,2.1,1.1,197.0,-46.7,-38.9,-46.6,-11.6,-29.7,-57.2,156.8,231.0,HNX,Construction & Materials,Price above SMA(20),Neutral,MACD Histogram < 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,NaN,NaN,NaN,NaN
3,EIB,,31858,8.4,2.3,2.5,3.4,2.5,0.0,1.4,34.8,29.9,10.5,0.6,18.1,1.5,NaN,0.0,54.76,13.74,205.12,20.38,NaN,19.58,NaN,138.00,181.64,172.98,58,59,20,47,101.60,18.30,50.5,2.77,46,-1675,2,NaN,1013,0.9,0.6,0.6,0.8,-2.14,2.81,0.55,2.81,20.52,32.13,90,NaN,NaN,1764.1,-26.7,-26.8,-76.4,-51.5,-18.2,-45.2,33.6,124.7,HOSE,Banks,Price below SMA(20),Sell,MACD Histogram < 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Price cross above SMA(5),Buy < Sell,NaN,NaN,NaN
4,HDB,,52644,22.2,3.8,4.4,3.8,3.1,0.1,0.8,57.3,41.5,6.9,1.1,6.2,1.2,NaN,7.6,31.08,23.96,28.02,25.04,NaN,41.33,NaN,113.34,119.74,155.82,58,47,86,71,121.26,18.20,50.9,19.62,66,10599,1,NaN,2926,1.4,1.1,0.9,0.8,-0.55,-0.82,1.11,-0.82,33.70,17.31,70,NaN,NaN,8463.3,12.8,14.9,23.0,3.8,-5.0,-39.2,41.0,117.6,HOSE,Banks,Price below SMA(20),Neutral,MACD Histogram < 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Price cross above SMA(5),Buy > Sell,NaN,NaN,NaN
